In [1]:
import requests
from bs4 import BeautifulSoup
import json

## 신규 출시 인기제품의 타이틀 따보기
- 크롤링 가닥을 잡기 위해 해본 것

In [ ]:
result = []
for start in range(0, 401, 50):
    URL = 'https://store.steampowered.com/search/results/?query&start='+str(start)+'&count=50&sort_by=Released_DESC&snr=1_7_7_popularnew_7&filter=popularnew&os=win&infinite=1'
    res = requests.get(URL, cookies={
        'Steam_Language':'koreana'
    })
    json_data = res.json()
    soup = BeautifulSoup(json_data['results_html'], 'html.parser')
    titles = soup.select('.col.search_name.ellipsis .title')
    
    for title in titles:
        result.append(title.text)

result

## 신규 출시 인기제품의 app id수집
- https://store.steampowered.com/app/+appid 해주면 각각의 게임들의 페이지에 접속할 수 있다.

In [2]:
# a태그의 appid들을 수집
# 기존의 아이디 리스트를 수집하는 과정
id_list = []
for start in range(0, 451, 50):
    URL = 'https://store.steampowered.com/search/results/?query&start='+str(start)+'&count=50&sort_by=Released_DESC&snr=1_7_7_popularnew_7&filter=popularnew&os=win&infinite=1'
    res = requests.get(URL, cookies={
        'Steam_Language':'koreana'
    })
    json_data = res.json()
    soup = BeautifulSoup(json_data['results_html'], 'html.parser')
    appids = soup.select('a')
    
    for appid in appids:
        idtag = appid.attrs['data-ds-appid']
        id_list.append(idtag)

In [ ]:
id_list

In [4]:
# 스팀은 아이디 규칙이 없다..
# 기존 id_list 파일 만들기 및 저장
with open('./id_list.txt', 'w') as f:
    for id in id_list:
        f.write(id + '\n')

### 신규 데이터를 체크할 때 id 값 비교
1. id값이 커지는 경우: 지금까지 저장한 id가져오고 이거보다 크면 신규데이터니까 가져오고 아니면 break, 그만 가져오기
2. id 규칙이 없는 경우: 최신순 정렬
    - 내가 크롤링한 데이터의 id를 모두 저장
    - `containment test` 진행

In [5]:
# 신작게임 많아봤자 3개나오니까 그냥 첫페이지만 긁어줌
new_crawling_id = []
URL = 'https://store.steampowered.com/search/results/?query&start=0&count=50&dynamic_data=&sort_by=Released_DESC&snr=1_7_7_popularnew_7&filter=popularnew&os=win&infinite=1'
res = requests.get(URL, cookies={
    'Steam_Language':'koreana'
})
json_data = res.json()
soup = BeautifulSoup(json_data['results_html'], 'html.parser')
appids = soup.select('a')[:10]

for appid in appids:
    idtag = appid.attrs['data-ds-appid']
    new_crawling_id.append(idtag)

In [6]:
new_crawling_id

['1636660', '1061910', '1726400', '1121640', '2060280']

In [7]:
# 기존 아이디 리스트를 불러온다.(\n이 포함돼있음)
with open('./id_list.txt', 'r', encoding='utf-8') as f:
    id_list = f.readlines()
    origin_id_list = [] # 공백 없애준 id를 담아주기위한 origin id list
    for id in id_list:
        temp = id.rstrip()
        origin_id_list.append(temp) 

In [9]:
new_crawling_id

['1636660', '1061910', '1726400', '1121640', '2060280']

In [10]:
# id list 갱신
new_id = []
for id in new_crawling_id:
    if id not in origin_id_list:
        new_id.append(id)
new_id

['1636660', '1061910']

# 수집한 app id를 바탕으로 각 게임 최소사양 불러오기

In [11]:
sysreqlist = {}
for i in range(len(new_id)):
    appidURL = 'https://store.steampowered.com/app/'+new_id[i]
    res = requests.get(appidURL, cookies={
        'Steam_Language':'koreana'
    })
    soup = BeautifulSoup(res.text, 'html.parser')

    sysreq_content = soup.select_one('.game_area_sys_req.sysreq_content.active[data-os="win"]')
    if sysreq_content.select_one('.game_area_sys_req_leftCol') == None:
        sysreqs = sysreq_content.select('.game_area_sys_req_full li')
    else:
         sysreqs = sysreq_content.select('.game_area_sys_req_leftCol li')
   
    appName = soup.select_one('#appHubAppName').text
    genre = soup.select('.glance_tags.popular_tags a')[0].text.strip()
    title = appName + ' (' + genre + ')'
    
    req_list = []
    for sysreq in sysreqs:
        # >문장<이 나오면 제외.
        if '필요합니다' in sysreq.text:
            continue
        elif '추가 사항' in sysreq.text:
            continue
        else:
            req = sysreq.text.strip()
            req_list.append(req)
    sysreqlist[title] = req_list

In [ ]:
sysreqlist

In [13]:
for id in origin_id_list:
    new_id.append(id)
# id_list 갱신
with open('./id_list.txt', 'w') as f:
    for id in new_id:
        f.write(id + '\n')

In [4]:
import os
currentPath = os.getcwd()
print(currentPath)
os.chdir(currentPath)

/home/ubuntu/seulki_package/Crawling


In [5]:
os.getcwd()

'/home/ubuntu/seulki_package/Crawling'